In [3]:
import sys
import os
import argparse

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [4]:
%run dataset.ipynb 
%run training.ipynb

if __name__ == '__main__':
    parser = argparse.ArgumentParser(usage='A training program for classifying the EMNIST dataset')
    parser.add_argument('-f', '--file', type=str, help='Path .mat file data', required=True)
    parser.add_argument('--width', type=int, default=28, help='Width of the images')
    parser.add_argument('--height', type=int, default=28, help='Height of the images')
    parser.add_argument('--max', type=int, default=None, help='Max amount of data to use')
    parser.add_argument('--epochs', type=int, default=10, help='Number of epochs to train on')
    parser.add_argument('--verbose', action='store_true', default=False, help='Enables verbose printing')

    #     args = parser.parse_args()

    data_file = 'emnist-byclass.mat'
    data_path = os.path.dirname(os.path.realpath(sys.argv[1])) + '/data/' + data_file
#     args = parser.parse_args(['-f', data_path, '--max', 10000])
    args = parser.parse_args(['-f', data_path])

    bin_dir = os.path.dirname(os.path.realpath(sys.argv[1])) + '/bin'
    if not os.path.exists(bin_dir):
        os.makedirs(bin_dir)

    training_data = load_data(args.file, width=args.width, height=args.height, max_data=args.max, verbose=args.verbose)
    model = build_net(training_data, width=args.width, height=args.height, epochs=args.epochs, verbose=args.verbose)


116323
Train on 697932 samples, validate on 116323 samples
Epoch 1/10
697932/697932 [==============================] - 2671s - loss: 0.7210 - acc: 0.7764 - val_loss: 0.4460 - val_acc: 0.8441
Epoch 2/10
697932/697932 [==============================] - 2626s - loss: 0.4855 - acc: 0.8344 - val_loss: 0.4071 - val_acc: 0.8556
Epoch 3/10
697932/697932 [==============================] - 2737s - loss: 0.4472 - acc: 0.8450 - val_loss: 0.3925 - val_acc: 0.8590
Epoch 4/10
697932/697932 [==============================] - 2963s - loss: 0.4289 - acc: 0.8499 - val_loss: 0.3931 - val_acc: 0.8589
Epoch 5/10
697932/697932 [==============================] - 2951s - loss: 0.4188 - acc: 0.8525 - val_loss: 0.3816 - val_acc: 0.8638
Epoch 6/10
697932/697932 [==============================] - 2538s - loss: 0.4113 - acc: 0.8550 - val_loss: 0.3735 - val_acc: 0.8644
Epoch 7/10
697932/697932 [==============================] - 2502s - loss: 0.4056 - acc: 0.8564 - val_loss: 0.3762 - val_acc: 0.8648
Epoch 8/10
697932